<a href="https://colab.research.google.com/github/arronelab/makeMolecules/blob/main/notebook/MakeMolecules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Clone Repo and Compile { display-mode: "form" }

#@markdown Please execute this cell by pressing the *Play* button on 
#@markdown the left.
%cd /content/
%shell rm -rf makeMolecules
!git clone https://github.com/arronelab/makeMolecules.git makeMolecules
%cd makeMolecules/
%shell sh makeFileFinalPrediction.sh

In [ ]:
#@title Install Requirements { display-mode: "form" }

#@markdown **Note**: This installs the software on the Colab 
#@markdown notebook in the cloud and not on your computer.
!pip install Bio

import urllib.request
import os
import sys
import re
import json
import numpy as np
import pandas as pd
import ast
from Bio import PDB, SeqIO
from Bio.SeqUtils import seq1 
from google.colab import files

In [ ]:
#@title Config File { display-mode: "form" }
#@markdown  First, choose a project name.
test_name = "test" #@param {type:"string"}
#@markdown Next, set the number of changes from the initial structure. This is changes to the whole structure, that is if you choose to make just one change, each atom that will be changed once, not just one atom.
number_changes = 1 #@param {type:"integer"}
#@markdown Finally, set the number of times you'd like to perform the test.  This is the number of output files you will get.
number_outputs =  4#@param {type:"integer"}
#@markdown Once you have set the parameters above, press the *Play* button on the left to make your config file.

def write_sh_file(project_name,fit_steps,no_mols,
                  preprocessed_data_dir = "newFitData/human_SMARCAL1",
                  fingerprint_path = "newFitData/human_SMARCAL1/fingerPrint.dat", 
                  coords_path = "newFitData/human_SMARCAL1/coordinates.dat", 
                  contacts_path = None, 
                  fixed_sections_path = None                 
                  ):
    """Writes the config shell script based on parameters given by the user on data upload.

    Args:
        project_name (str): [description]
        preprocessed_data_dir (str): [description]
        fingerprint_path (str): [description]
        coords_path (str, optional): [description]. Defaults to None.
        contacts_path (str, optional): [description]. Defaults to None.
        fixed_sections_path (str, optional): [description]. Defaults to None.
        fit_steps (int, optional): [description]. Defaults to 1000.

    Returns:
        output_filepath (str): path to the finished shell script
    """
    output_filepath = project_name + '_config.sh'
    output_dir = os.path.join(preprocessed_data_dir,project_name)
    with open(output_filepath, 'w+') as fout:
      fout.write('#!/bin/bash')
      fout.write('\nSequenceFile={}'.format(fingerprint_path))
      if coords_path is not None:
        fout.write('\ninitialCoordsFile={}'.format(coords_path))
      else:
        fout.write('\ninitialCoordsFile=none')
      if contacts_path is not None:
          fout.write('\npairedPredictions={}'.format(contacts_path))
      else:
          fout.write('\npairedPredictions=none')
      if fixed_sections_path is not None:
          fout.write('\nfixedsections={}'.format(fixed_sections_path))
      else:
          fout.write('\nfixedsections=none')
        # no options for crystal symmetry or or hydro cover in prototype
      fout.write('\ncrystalSymmetry=none')
      fout.write('\nwithinMonomerHydroCover=none')
      fout.write('\nbetweenMonomerHydroCover=none')
      fout.write('\nmaxNoFitSteps={}'.format(str(fit_steps)))
      fout.write('\n\nif [ ! -d {} ]; then'.format(output_dir))
      fout.write('\n  mkdir -p {};'.format(output_dir))
      fout.write('\nfi')
      fout.write('\n\nfor i in {1..'+str(no_mols)+'}')
      fout.write('\ndo')
      fout.write('\n    /content/makeMolecules/predictStructure $SequenceFile '
    	'$initialCoordsFile $pairedPredictions $fixedsections $crystalSymmetry '
    	'$withinMonomerHydroCover $betweenMonomerHydroCover '
    	'$maxNoFitSteps /content/makeMolecules/{}/mol$i.dat'.format(output_dir))
      fout.write('\ndone')
    return output_filepath

if __name__ == "__main__":
    write_sh_file(project_name=test_name,fit_steps=number_changes,no_mols=number_outputs)
    

In [ ]:
#@title Make Molecules { display-mode: "form" }

#@markdown Please execute this cell by pressing the *Play* button on 
#@markdown the left. Your outputs will be in a folder with your chosen test name, inside the newFitData/human_SMARCAL1 tree.
outputs_dir = '/content/makeMolecules/newFitData/human_SMARCAL1/'+test_name
!bash {test_name}_config.sh


In [ ]:
#@title Download Outputs { display-mode: "form" }

#@markdown Press the *Play* button if you'd like to download your outputs in a
#@markdown zip file.
%cd /content/makeMolecules/newFitData/human_SMARCAL1/
!zip -q -r {outputs_dir}.zip {test_name}
files.download(f'{outputs_dir}.zip')
%cd /content/makeMolecules/